In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
from pycaret.classification import *
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

In [ ]:
df = pd.read_csv("../input/adult-data-set-uci-ml/Adult Data Set.csv")

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df["education-num"] = df["education-num"].fillna(df["education-num"].median())

In [ ]:
df.corr()

In [ ]:
random = []
for i in range(488):
    random.append(np.random.randint(1,177))
def change(x):
    x=np.random.randint(1,177)
    return country_list[x]


In [ ]:
occ_list = list(df["occupation"])
occ_list = list(np.unique(occ_list))
occ_list.remove("nan")

In [ ]:
random = []
def change(x):
    x=np.random.randint(1,15)
    return occ_list[x]
occu_null=df[df['occupation'].isnull()]['occupation'].map(lambda x:change(x))
occu_null

In [ ]:
occu_notnull=df[df['occupation'].isnull()!=True]['occupation']
occu_null_df=pd.DataFrame(occu_null)
occu_notnull_df=pd.DataFrame(occu_notnull)

In [ ]:
df_occu = pd.concat([occu_null_df,occu_notnull_df],axis = 0)
df["occupation"]=df_occu.sort_index(axis=0)

In [ ]:
df.isnull().sum()

In [ ]:
def one_hot_encoding(df,col):
    one_hot_encoder=ce.OneHotEncoder(cols=col,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final

In [ ]:
df = one_hot_encoding(df,"workclass")
df = one_hot_encoding(df,"education")
df = one_hot_encoding(df,"marital-status")
df = one_hot_encoding(df,"occupation")
df = one_hot_encoding(df,"relationship")
df = one_hot_encoding(df,"race")
df = one_hot_encoding(df,"native-country")

In [ ]:
def ordinal_encoding(df,col,mapping):
    ordinal_encoder=ce.OrdinalEncoder(cols=col,return_df=True,mapping=[{'col':col,'mapping':mapping}])
    df_final = ordinal_encoder.fit_transform(df)
    return df_final

In [ ]:
df = ordinal_encoding(df,"sex",{"Male":1,"Female":0})

In [ ]:
df = ordinal_encoding(df,"salary",{">=50k":60000,"<50k":40000})

In [ ]:
df

In [ ]:
X = df.drop("salary",axis = 1)
Y = df["salary"]
smote = SMOTE()
X, Y = smote.fit_resample(X, Y)
df = pd.DataFrame(X)
df["salary"] = Y

In [ ]:
experiment = setup(df, target="salary",normalize = True,use_gpu = True)

In [ ]:
compare_models()

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 27)

In [ ]:
X_train = normalize(X_train)
X_test = normalize(X_test)

In [ ]:
model = LGBMClassifier()

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
accuracy = model.score(X_test, Y_test)

In [ ]:
accuracy_score(Y_pred,Y_test)

In [ ]:
df.dtypes